<a href="https://colab.research.google.com/github/mccoymb/AAE-590-DSMM/blob/main/590DSMM_HW6_3_big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# This was run in Jupyter Lab, will not work in colab
import numpy as np
import cv2
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr

time_values = np.array([25000, 28000, 31000, 35000, 37500, 40000, 45000, 50000, 75000, 108000,
                        180000, 220000, 250000, 270000, 300000, 360000, 430000, 540000, 900000, 1080000])
log_time = np.log(time_values)

In [8]:
data = np.load('/home/mccoymb/Downloads/train.npz')

print("Keys in npz file:", data.files)
input_raw_data = data['input_raw_data']  # Shape: (200000, 1, 64, 64)

FileNotFoundError: [Errno 2] No such file or directory: '/home/mccoymb/Downloads/train.npz'

In [2]:
log_time = np.log(time_values)

# Function to compute grain size
def compute_grain_size(image):
    _, binary = cv2.threshold((image * 255).astype(np.uint8), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours if cv2.contourArea(cnt) > 5]
    return np.mean(areas) if areas else 1e-6

# Final results list
results = []

# Loop through different sequence sizes
for num_sequences in [10, 50, 100]:
    subset_raw_data = input_raw_data[:num_sequences * 20]
    all_grain_sizes = np.zeros((num_sequences, 20))

    for seq in range(num_sequences):
        imgs = subset_raw_data[seq * 20:(seq + 1) * 20, 0, :, :]
        grain_sizes = np.array([compute_grain_size(img) for img in imgs])
        first = max(grain_sizes[0], 1)
        all_grain_sizes[seq] = grain_sizes / first

    # Flatten and filter
    X = np.tile(log_time, num_sequences)
    y = all_grain_sizes.flatten()

    # Outlier removal
    Q1, Q3 = np.percentile(y, [25, 75])
    IQR = Q3 - Q1
    mask = (y >= Q1 - 1.5 * IQR) & (y <= Q3 + 1.5 * IQR)
    X_filtered = X[mask].reshape(-1, 1)
    y_filtered = y[mask]

    # --- GPR ---
    kernel = C(1.0, (1e-2, 1e2)) * RBF(1.0, (1e-2, 1e2))
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=0.01, n_restarts_optimizer=10)
    gpr.fit(X_filtered, y_filtered)
    y_pred_gpr = gpr.predict(X_filtered)
    r2 = r2_score(y_filtered, y_pred_gpr)
    adj_r2 = 1 - (1 - r2) * (len(y_filtered) - 1) / (len(y_filtered) - 2)
    mse = mean_squared_error(y_filtered, y_pred_gpr)
    r, _ = pearsonr(y_filtered, y_pred_gpr)
    cv_r2 = np.mean(cross_val_score(gpr, X_filtered, y_filtered, cv=5, scoring='r2'))

    results.append({
        "Model": "GPR",
        "Sequences": num_sequences,
        "R²": round(r2, 4),
        "Adj. R²": round(adj_r2, 4),
        "MSE": round(mse, 6),
        "Pearson R": round(r, 4),
        "CV R²": round(cv_r2, 4)
    })

    # --- BLR ---
    blr = BayesianRidge()
    blr.fit(X_filtered, y_filtered)
    y_pred_blr = blr.predict(X_filtered)
    r2 = r2_score(y_filtered, y_pred_blr)
    adj_r2 = 1 - (1 - r2) * (len(y_filtered) - 1) / (len(y_filtered) - 2)
    mse = mean_squared_error(y_filtered, y_pred_blr)
    r, _ = pearsonr(y_filtered, y_pred_blr)
    cv_r2 = np.mean(cross_val_score(blr, X_filtered, y_filtered, cv=5, scoring='r2'))

    results.append({
        "Model": "Bayesian Ridge",
        "Sequences": num_sequences,
        "R²": round(r2, 4),
        "Adj. R²": round(adj_r2, 4),
        "MSE": round(mse, 6),
        "Pearson R": round(r, 4),
        "CV R²": round(cv_r2, 4)
    })

# ----- Export Table -----
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

NameError: name 'input_raw_data' is not defined